In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *

from visualization_utilities import *

import time

from multiprocess import Pool

Setting environment for Gordon


No vtk


In [22]:
stack = 'MD589'

In [26]:
setting = 2
# add_annotation = True
downscale = 8

In [77]:
c = np.array(random_colors(3)[0])/255.

In [94]:
sec = 232
viz = scoremap_overlay(stack=stack, sec=sec, structure='10N', downscale=downscale, setting=2, color=(1,1,0))

In [95]:
display_image(viz, filename='tmp.png')

/oasis/projects/nsf/csd395/yuncong/Brain/learning/tmp.png

In [23]:
# for sec in range(200, 202):
for sec in [201]:
    
    t = time.time()
    
    bg = imread(DataManager.get_image_filepath(stack=stack, section=sec, resol='lossless', version='compressed'))
    
    def f(structure):
        viz_fp = DataManager.get_scoremap_viz_filepath(stack=stack, section=sec, structure=structure, setting=setting)
        try:
            viz = scoremap_overlay_on(bg=bg, stack=stack, sec=sec, structure=structure, downscale=downscale, label_text=None, setting=setting)
            create_if_not_exists(os.path.dirname(viz_fp))
            imsave(viz_fp, img_as_ubyte(viz))
        except Exception as e:
            sys.stderr.write('%s\n' % e)
            return
    
    pool = Pool(8)
    pool.map(f, all_known_structures)
    pool.close()
    pool.join()

    sys.stderr.write('Visualize scoremaps: %.2f seconds.\n' % (time.time() - t)) 
    # 7s for one structure, one section, single process
    # 20s for all structures, one section, 8 processes

/oasis/projects/nsf/csd395/yuncong/Brain/utilities/visualization_utilities.py:131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if bg == 'original':
/oasis/projects/nsf/csd395/yuncong/Brain/utilities/visualization_utilities.py:131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if bg == 'original':
/oasis/projects/nsf/csd395/yuncong/Brain/utilities/visualization_utilities.py:131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if bg == 'original':
/oasis/projects/nsf/csd395/yuncong/Brain/utilities/visualization_utilities.py:131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if bg == 'original':
/oasis/projects/nsf/csd395/yuncong/Brain/utilities/visualization_utiliti

In [ ]:
# if not add_annotation:
# export_scoremaps_multiprocess(bg='original', stack=stack, sections=range(200, 201),
#                               structures=all_known_structures, downscale_factor=8,
#                               label_text=True, setting=setting) # 93s/section
# else:
#     outputViz_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_scoremapPlusAnnotationViz'
#     export_filepath_fmt = outputViz_rootdir + '/%(name)s/%(stack)s/%(fn)s_alignedTo_%(anchor_fn)s_scoremapPlusAnnotationViz_%(name)s_%(annofn)s.jpg' % dict(anchor_fn=anchor_fn)
#     export_scoremapPlusAnnotationVizs('original', stack, range(first_sec, last_sec+1),
#                                       structures, 8, export_filepath_fmt=export_filepath_fmt,
#                                       users=['yuncong', 'localAdjusted', 'autoAnnotate', 'globalAligned'])
